In [55]:
import os
#from dotenv import load_dotenv
#load_dotenv()
from sqlalchemy.sql import func
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
from sqlalchemy import Column, Integer, String, Float
import numpy as np
import pandas as pd


#################################################
# Database Setup
#################################################
engines = create_engine("sqlite:///Datasets//Weather_3.db")

# Save reference to the table

conn = engines.connect()
Base = automap_base()
Base.prepare(engines, reflect=True)
session = Session(conn)
Base.classes.keys()
# Performs database schema inspection
#insp = sqlalchemy.inspect(engines)
#print(insp.get_table_names())
Wtr = Base.classes.Weather_Raw
St_Cd = Base.classes.State_Code
#join = session.query( Wtr , St_Cd ).filter(Wtr.State == St_Cd.Code).statement
sel = [St_Cd.State, Wtr.Year, Wtr.Jan, Wtr.Feb, Wtr.Mar, Wtr.Apr, Wtr.May, Wtr.Jun, Wtr.Jul, Wtr.Aug, Wtr.Sep, Wtr.Oct, Wtr.Nov, Wtr.Dec]
avg = [Wtr.Jan, Wtr.Feb, Wtr.Mar, Wtr.Apr, Wtr.May, Wtr.Jun, Wtr.Jul, Wtr.Aug, Wtr.Sep, Wtr.Oct, Wtr.Nov, Wtr.Dec]
# This the  same list I was using for the Pandas dataframe but instead I am passing in the session query 
States_list = [2, 3, 4, 8, 11, 18, 30, 45, 41,42] 
result = session.query(St_Cd.State, Wtr.Year, func.round(func.avg(Wtr.Jan), 2).label('Jan'), func.round(func.avg(Wtr.Feb), 2).label('Feb'), func.round(func.avg(Wtr.Mar), 2).label('Mar'),\
                                              func.round(func.avg(Wtr.Apr), 2).label('Apr'), func.round(func.avg(Wtr.May), 2).label('May'), func.round(func.avg(Wtr.Jun), 2).label('Jun'),\
                                              func.round(func.avg(Wtr.Jul), 2).label('Jul'), func.round(func.avg(Wtr.Aug), 2).label('Aug'), func.round(func.avg(Wtr.Sep), 2).label('Sep'),
                                              func.round(func.avg(Wtr.Oct), 2).label('Oct'), func.round(func.avg(Wtr.Nov), 2).label('Nov'), func.round(func.avg(Wtr.Dec), 2).label('Dec'))\
.filter(Wtr.State == St_Cd.Code).filter(Wtr.State.in_(States_list)) \
.filter((Wtr.Year > 2010) & (Wtr.Year < 2022 )).group_by(St_Cd.State, Wtr.Year).statement
df = pd.read_sql_query(result, session.bind)
print(df)

# States_list = [2, 3, 4, 8, 11, 18, 30, 45, 41,42]
# df1 = df.loc[((df['Year'] >= 2012) & (df['Year'] <= 2021) & (df['State'].isin(States_list)))] ### this is where put the 10 states and the years that we are planning to use
# df1 = df1.drop(['Element' , 'County', 'Code' , 'State'] ,  axis=1)

# df_avg = round(df1.groupby(['State_1', 'Year']).mean(), 2)
#print(df_avg)
# Sending output to a CSV file to get the full picture
#df.to_csv(r'C:\Users\georg\project-3-group-1\Datasetsr', index=True, header=True)
session.close()


          State  Year    Jan    Feb    Mar    Apr    May    Jun    Jul    Aug  \
0       Arizona  2011  43.43  43.47  55.13  60.85  64.73  77.20  81.78  83.10   
1       Arizona  2012  47.07  47.43  52.98  62.06  70.54  79.83  80.59  81.37   
2       Arizona  2013  40.70  43.82  56.39  60.93  68.75  80.61  81.56  79.45   
3       Arizona  2014  48.32  51.81  55.49  60.61  68.13  79.08  82.02  77.40   
4       Arizona  2015  46.73  53.59  57.71  59.79  64.23  79.58  79.49  81.71   
..          ...   ...    ...    ...    ...    ...    ...    ...    ...    ...   
105  Washington  2017  26.99  33.07  40.70  45.19  54.17  59.78  66.76  68.45   
106  Washington  2018  37.21  34.04  40.02  46.29  57.76  58.59  67.51  66.15   
107  Washington  2019  35.96  27.44  37.81  47.24  56.03  59.79  64.02  66.48   
108  Washington  2020  36.15  36.90  39.34  46.89  53.99  58.27  64.69  65.71   
109  Washington  2021  36.26  33.01  40.64  47.61  53.22  64.76  69.19  65.94   

       Sep    Oct    Nov   

In [ ]:
df_htm = df_avg.to_html()

In [56]:
from operator import index
import os
#from dotenv import load_dotenv
#load_dotenv()
from sqlalchemy.sql import func
from sqlalchemy import create_engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
#import matplotlib.pyplot as plt
#from sqlalchemy import Column, Integer, String, Float
import numpy as np
import pandas as pd

#Below Flask Import added by Prina##
from flask import (
    Flask,
    render_template,
    jsonify,
    request,
    redirect)

#################################################
# Database Setup
#################################################
engines = create_engine("sqlite:///Datasets//Weather_3.db")

# Save reference to the table

conn = engines.connect()
Base = automap_base()
Base.prepare(engines, reflect=True)
session = Session(conn)
Base.classes.keys()
# Performs database schema inspection
#insp = sqlalchemy.inspect(engines)
#print(insp.get_table_names())
Wtr = Base.classes.Weather_Raw
St_Cd = Base.classes.State_Code

# This the  same list I was using for the Pandas dataframe but instead I am passing in the session query 
# States_list = [2, 3, 4, 8, 11, 18, 30, 45, 41,42] 
# result = session.query(St_Cd.State, Wtr.Year, func.round(func.avg(Wtr.Jan), 2).label('Jan'), func.round(func.avg(Wtr.Feb), 2).label('Feb'), func.round(func.avg(Wtr.Mar), 2).label('Mar'),\
#                                               func.round(func.avg(Wtr.Apr), 2).label('Apr'), func.round(func.avg(Wtr.May), 2).label('May'), func.round(func.avg(Wtr.Jun), 2).label('Jun'),\
#                                               func.round(func.avg(Wtr.Jul), 2).label('Jul'), func.round(func.avg(Wtr.Aug), 2).label('Aug'), func.round(func.avg(Wtr.Sep), 2).label('Sep'),
#                                               func.round(func.avg(Wtr.Oct), 2).label('Oct'), func.round(func.avg(Wtr.Nov), 2).label('Nov'), func.round(func.avg(Wtr.Dec), 2).label('Dec'))\
# .filter(Wtr.State == St_Cd.Code).filter(Wtr.State.in_(States_list)) \
# .filter((Wtr.Year > 2010) & (Wtr.Year < 2022 )).group_by(St_Cd.State, Wtr.Year).statement
# df = pd.read_sql_query(result, session.bind)
# print(df)

join = session.query( Wtr , St_Cd ).filter(Wtr.State == St_Cd.Code).statement
df = pd.read_sql_query(join, session.bind)
print(df)
States_list = [2, 3, 4, 8, 11, 18, 30, 45, 41,42]
df1 = df.loc[((df['Year'] >= 2012) & (df['Year'] <= 2021) & (df['State'].isin(States_list)))] ### this is where put the 10 states and the years that we are planning to use
df1 = df1.drop(['Element' , 'County', 'Code' , 'State'] ,  axis=1)
df_avg = round(df1.groupby(['State_1', 'Year']).mean(), 2)
#print(df_avg)
# Sending output to a CSV file to get the full picture
#df.to_csv(r'C:\Users\georg\project-3-group-1\Datasetsr', index=True, header=True)
session.close()

        State  County  Element  Year   Jan   Feb   Mar   Apr   May   Jun  \
0           1       1        2  1895  44.0  38.2  55.5  64.1  70.6  78.3   
1           1       1        2  1896  44.3  49.0  54.0  69.3  76.8  78.0   
2           1       1        2  1897  43.7  52.3  61.3  63.0  70.0  82.4   
3           1       1        2  1898  50.1  46.8  60.1  59.6  75.0  81.5   
4           1       1        2  1899  44.6  41.5  56.6  62.3  76.7  81.0   
...       ...     ...      ...   ...   ...   ...   ...   ...   ...   ...   
400631     50     290        2  2018  -2.2   6.1  15.1  25.6  43.6  54.6   
400632     50     290        2  2019  -1.8   9.4  25.0  29.2  47.0  59.3   
400633     50     290        2  2020 -16.2  -7.4   7.9  27.3  47.7  55.9   
400634     50     290        2  2021   3.6  -9.0   4.4  22.9  44.3  56.3   
400635     50     290        2  2022  -4.8 -99.9 -99.9 -99.9 -99.9 -99.9   

         Jul   Aug   Sep   Oct   Nov   Dec  Code  State_1  
0       80.4  80.4  79.0  6